In [19]:
import data
import classifier
import utils

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import pandas as pd

pd.set_option('display.max_columns', None)


In [20]:

PATH_TO_CSV = '/home/firl/bot-detector-ml-training/src/data/raw/2022-10-26_hiscore_data.csv'
PATH_TO_CSV = None
if PATH_TO_CSV is None:
    print('set PATH_TO_CSV=/path/to/2022-10-26_hiscore_data.csv and run all')
    raise ValueError

df = pd.read_csv(PATH_TO_CSV)

In [21]:
# This logic is roughly equivalent to app.py's train function
hiscoredata = data.hiscoreData(df, False)
features = hiscoredata.features()
# no playerData()-related data, its already in features
features_labeled = features
binary_classifier = classifier.classifier("binaryClassifier")
# Logic should be the same as app.py line 179: train the model 
x = features_labeled.copy()
y = x['label'].apply(lambda x: 0 if x == 'Real_Player' else 1)
# TODO: one-hot encode account status
print('unique account_status values:', x['account_status'].unique())
# TODO: using created_at, updated_at epoch times slightly improve binary classifier
# x['created_at'] = pd.to_datetime(x['created_at'], format="%Y-%m-%d %H:%M:%S").apply(lambda x: (x - pd.Timestamp("1970-01-01")) / pd.Timedelta("1s"))
# x['updated_at'] = pd.to_datetime(x['updated_at'], format="%Y-%m-%d %H:%M:%S").apply(lambda x: (x - pd.Timestamp("1970-01-01")) / pd.Timedelta("1s"))
# print(x['created_at'].head())
x.drop(columns=['label', 'label_id', 'name', 'created_at', 'updated_at', 'account_status', 'possible_ban', 'confirmed_ban'], inplace=True)
print('x columns:\n', x.columns)
print('x head:\n', x.head())
print('y head:\n', y.head())
x.describe()


unique account_status values: ['not banned' 'banned']
x columns:
 Index(['total', 'attack', 'defence', 'strength', 'hitpoints', 'ranged',
       'prayer', 'magic', 'cooking', 'woodcutting',
       ...
       'tombs_of_amascut/total', 'tombs_of_amascut_expert/total',
       'tzkal_zuk/total', 'tztok_jad/total', 'venenatis/total', 'vetion/total',
       'vorkath/total', 'wintertodt/total', 'zalcano/total', 'zulrah/total'],
      dtype='object', length=160)
x head:
                total    attack   defence  strength  hitpoints    ranged  \
Player_id                                                                 
1           55978987   5491118   3358410   5789019    5237822   1877958   
8          249164755  14151880  13293654  15162818   25533323  19180307   
29          25751694    674007         0   3150582    2971492   2595141   
39         330938656  14153462  13409025  14231093   23612757  20071372   
59         380812843  25789847  19279215  57396170   51308981  45806374   

      

,total,attack,defence,strength,hitpoints,ranged,prayer,magic,cooking,woodcutting,fletching,fishing,firemaking,crafting,smithing,mining,herblore,agility,thieving,slayer,farming,runecraft,hunter,construction,league,bounty_hunter_hunter,bounty_hunter_rogue,cs_all,cs_beginner,cs_easy,cs_medium,cs_hard,cs_elite,cs_master,lms_rank,soul_wars_zeal,abyssal_sire,alchemical_hydra,barrows_chests,bryophyta,callisto,cerberus,chambers_of_xeric,chambers_of_xeric_challenge_mode,chaos_elemental,chaos_fanatic,commander_zilyana,corporeal_beast,crazy_archaeologist,dagannoth_prime,dagannoth_rex,dagannoth_supreme,deranged_archaeologist,general_graardor,giant_mole,grotesque_guardians,hespori,kalphite_queen,king_black_dragon,kraken,kreearra,kril_tsutsaroth,mimic,nex,nightmare,phosanis_nightmare,obor,sarachnis,scorpia,skotizo,tempoross,the_gauntlet,the_corrupted_gauntlet,theatre_of_blood,theatre_of_blood_hard,thermonuclear_smoke_devil,tombs_of_amascut,tombs_of_amascut_expert,tzkal_zuk,tztok_jad,venenatis,vetion,vorkath,wintertodt,zalcano,zulrah,boss_total,attack/total,defence/total,strength/total,hitpoints/total,ranged/total,prayer/total,magic/total,cooking/total,woodcutting/total,fletching/total,fishing/total,firemaking/total,crafting/total,smithing/total,mining/total,herblore/total,agility/total,thieving/total,slayer/total,farming/total,runecraft/total,hunter/total,construction/total,abyssal_sire/total,alchemical_hydra/total,barrows_chests/total,bryophyta/total,callisto/total,cerberus/total,chambers_of_xeric/total,chambers_of_xeric_challenge_mode/total,chaos_elemental/total,chaos_fanatic/total,commander_zilyana/total,corporeal_beast/total,crazy_archaeologist/total,dagannoth_prime/total,dagannoth_rex/total,dagannoth_supreme/total,deranged_archaeologist/total,general_graardor/total,giant_mole/total,grotesque_guardians/total,hespori/total,kalphite_queen/total,king_black_dragon/total,kraken/total,kreearra/total,kril_tsutsaroth/total,mimic/total,nex/total,nightmare/total,phosanis_nightmare/total,obor/total,sarachnis/total,scorpia/total,skotizo/total,tempoross/total,the_gauntlet/total,the_corrupted_gauntlet/total,theatre_of_blood/total,theatre_of_blood_hard/total,thermonuclear_smoke_devil/total,tombs_of_amascut/total,tombs_of_amascut_expert/total,tzkal_zuk/total,tztok_jad/total,venenatis/total,vetion/total,vorkath/total,wintertodt/total,zalcano/total,zulrah/total
count,2.862340e+05,2.862340e+05,2.862340e+05,2.862340e+05,2.862340e+05,2.862340e+05,2.862340e+05,2.862340e+05,2.862340e+05,2.862340e+05,2.862340e+05,2.862340e+05,2.862340e+05,2.862340e+05,2.862340e+05,2.862340e+05,2.862340e+05,2.862340e+05,2.862340e+05,2.862340e+05,2.862340e+05,2.862340e+05,2.862340e+05,2.862340e+05,286234.0,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.000000,286234.00000,286234.0,286234.0,286234.0,286234.000000,286234.0,286234.0,286234.0,286234.0,286

In [22]:
# This logic is roughly equivalent to app.py's train function
train_x, test_x, train_y, test_y = train_test_split(
    x, y, test_size=0.2, random_state=42, stratify=y
)
binary_classifier.fit(train_x, train_y)


classifier(name='binaryClassifier',
           path='/home/firl/bot-detector-ML/api/MachineLearning/models')

In [23]:
print(binary_classifier.score(test_y, test_x))
# OUTPUT: (0.9992126580557206, 0.9992126580557205)

(0.9752825054161562, 0.9752825054161562)


In [24]:
print(
    classification_report(
        y_true=test_y, 
        y_pred=binary_classifier.predict(test_x)
    )
)

              precision    recall  f1-score   support

           0       0.98      0.97      0.97     26765
           1       0.97      0.98      0.98     30482

    accuracy                           0.98     57247
   macro avg       0.98      0.98      0.98     57247
weighted avg       0.98      0.98      0.98     57247



In [25]:
# pd.to_datetime("2010/11/12", format="%Y/%m/%d")
# Out[52]: Timestamp('2010-11-12 00:00:00')

# pd.to_datetime("12-11-2010 00:00", format="%d-%m-%Y %H:%M")
# Out[53]: Timestamp('2010-11-12 00:00:00')

# TODO: play around with updated_at/created_at and other feature extraction
# # print(features_labeled['created_at'].dtypes)
# # created_at_ts = pd.to_datetime(features_labeled['created_at'], format="%Y-%m-%d %H:%M:%S")
# # updated_at_ts = pd.to_datetime(features_labeled['updated_at'], format="%Y-%m-%d %H:%M:%S")
# # acct_lifespan_ts = updated_at_ts - created_at_ts
# # print(created_at_ts.head())
# # print(updated_at_ts.head())
# # print(acct_lifespan_ts)

# created_at_ts.apply(lambda x: (x - pd.Timestamp("1970-01-01")) / pd.Timedelta("1s")).head()
# (stamps - pd.Timestamp("1970-01-01")) // pd.Timedelta("1s")
# pd.to_datetime(features_labeled['created_at'], format="%Y-%m-%d %H:%M:%S").dtypes

In [29]:
# pca code loosely based on:
# https://scikit-learn.org/stable/auto_examples/decomposition/plot_pca_iris.html

def do_pca(X, standardize=True, n_components=2):
    pca = PCA(n_components=n_components)
    if standardize:
        estimator = make_pipeline(StandardScaler(), pca).fit(X)
        X_r = estimator.transform(X)
    else:
        X_r = pca.fit(X).transform(X)
    print("explained variance ratio: %s" % str(pca.explained_variance_ratio_))
    return X_r
    
def plot_2d_pca(X, y, omit_real=True, make_bad_plot=False, standardize=True):
    y = y.copy()
    y = pd.DataFrame(y)
    print(y.head())
    X_r = do_pca(X, standardize)
    
    if omit_real:
        X_r = X_r[y['label'] != 'Real_Player']
        y = y[y['label'] != 'Real_Player']
    
    if make_bad_plot:
        fig, ax = plt.subplots(figsize=(1,1), dpi=80)
        lw = 1
        #for color, i in zip(colors, [0, 1, 2]):
        label_to_id = dict((label, label_id) for label_id, label in enumerate(y['label'].unique()))
        ax.scatter(X_r[:,0], X_r[:,1], c=y['label'].apply(lambda l: label_to_id[l]), alpha=0.3, norm='log', lw=lw, label=y['label'])
        for i, label in enumerate(y['label'].values):
            ax.annotate(label, (X_r[i,0], X_r[i,1]), fontsize=4)
        ax.set_xscale('log')
        ax.set_yscale('log')
        ax.legend(loc="best", shadow=False, scatterpoints=1)
        # ax.title("PCA of IRIS dataset")
    
    
do_pca(x, standardize=True, n_components=len(train_x.columns))
plot_2d_pca(x, y)

explained variance ratio: [1.83177105e-01 3.69310075e-02 2.53014508e-02 2.33760639e-02
 1.82840809e-02 1.62430025e-02 1.35168292e-02 1.31922040e-02
 1.24127082e-02 1.19425441e-02 1.10165264e-02 1.09147723e-02
 1.06873877e-02 9.95494765e-03 9.75296090e-03 9.67289747e-03
 9.44670769e-03 9.13762024e-03 8.98248128e-03 8.72151227e-03
 8.55740902e-03 8.37952119e-03 8.28873444e-03 8.14575845e-03
 8.02426693e-03 7.96585535e-03 7.88329791e-03 7.81285742e-03
 7.53760875e-03 7.50866236e-03 7.43969309e-03 7.39265599e-03
 7.34585887e-03 7.29916453e-03 7.22453767e-03 7.15610133e-03
 7.09025210e-03 7.00556379e-03 6.96831273e-03 6.96390342e-03
 6.85455730e-03 6.76930990e-03 6.73110254e-03 6.66162951e-03
 6.62458593e-03 6.61407933e-03 6.59287373e-03 6.51680838e-03
 6.46258490e-03 6.43876522e-03 6.42397359e-03 6.39138795e-03
 6.36318708e-03 6.31538084e-03 6.29001848e-03 6.28172102e-03
 6.27143546e-03 6.22110631e-03 6.18624574e-03 6.14484851e-03
 6.09293852e-03 6.08187943e-03 5.92119953e-03 5.88209030e-0